In [28]:
# 워드 클라우드 생성 데이터 수집 & 파일 생성한 코드입니다.

from selenium import webdriver
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd

# 검색어 설정 : 하이퍼클로바X, 클로바X, 네이버 큐(CUE)
keyword = ['하이퍼클로바X', '네이버 하이퍼클로바X', '클로바X', '네이버 클로바X', '네이버 큐', '네이버 CUE']

news_title = []

for word in keyword:
    # 크롤링 기간 설정 (231119 기준 8일 전 ~ 1일 전)
    before_one_week = (datetime.now() - timedelta(8)).date().strftime('%Y-%m-%d').replace('-', ".")
    yesterday = (datetime.now() - timedelta(1)).date().strftime('%Y-%m-%d').replace('-', ".")

    crowl_url = f"https://search.naver.com/search.naver?where=news&query={word}&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds={before_one_week}&de={yesterday}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20231105to20231105&is_sug_officeid=0&office_category=0&service_area=0"

    # 브라우저 열기 & 웹페이지 접속
    browser = webdriver.Chrome()
    browser.get(crowl_url)
    time.sleep(2)

    soup = BeautifulSoup(browser.page_source, "html.parser")

    atit = soup.select("div.news_contents a.news_tit")
    for t in atit:
        title = t['title']
        news_title.append(title)


    # = 다음 페이지가 존재하는 경우
    # = is_next_page 값이 'false'인 경우
    is_next_page = soup.select("a.btn_next")[0]['aria-disabled']
    btn_next = browser.find_elements("css selector", "a.btn_next")[0]

    while True:

        if is_next_page == 'false':
            btn_next.click()
            time.sleep(1)

            soup = BeautifulSoup(browser.page_source, "html.parser")
            atit = soup.select("div.news_contents a.news_tit")
            for t in atit:
                title = t['title']
                news_title.append(title)

            is_next_page = soup.select("a.btn_next")[0]['aria-disabled']
            btn_next = browser.find_elements("css selector", "a.btn_next")[0]

        elif is_next_page == 'true':
            break
        else:
            break

# 겹치는 기사는 제거
unique_list = []
[unique_list.append(t) for t in news_title if t not in unique_list]

# 크롤링한 데이터 저장하기
df = pd.DataFrame(unique_list)
df.columns = ['title']
df.to_excel(f'./{before_one_week}_{yesterday}_AItitle.xlsx', index = False)